# Join the respective table to integrate the Product dimension table

In [0]:
from pyspark.sql.functions import current_timestamp

query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY pn.start_date, pn.product_number) AS product_key, -- Surrogate key
    pn.product_id,
    pn.product_number,
    pn.product_name,
    pn.category_id,
    pc.category,
    pc.subcategory,
    pc.maintenance_flag,
    pn.product_line,
    pn.start_date
FROM dev_project.silver.crm_prd_info pn
LEFT JOIN dev_project.silver.erp_px_cat_g1v2 pc
    ON pn.category_id = pc.category_id
--WHERE pn.end_date IS NULL; -- Filter out all historical data
"""
df = spark.sql(query)

In [0]:
df = df.withColumn("_ingest_time", current_timestamp())

In [0]:
df.display()

# write product table into gold

In [0]:
df.write.mode("overwrite").saveAsTable("dev_project.gold.gold_dim_products") 

In [0]:
%sql
SELECT * FROM dev_project.gold.gold_dim_products LIMIT 10